### Trimming Responses of Excess Output

In [18]:
import re
def extract_main_class(code: str, few_shot: bool) -> str:
    if few_shot:
        n = 2

    elif not few_shot:
        n = 1

    starts = [m.start() for m in re.finditer(r'public class Main', code)]

    if len(starts) < n:
        return ""  

    start_index = starts[n - 1]  # 0-based indexing
    brace_count = 0
    in_class = False
    end_index = None
    num_complete = 0

    for i in range(start_index, len(code)):
        char = code[i]

        if char == '{':
            brace_count += 1
            if not in_class:
                in_class = True
        elif char == '}':
            brace_count -= 1
            if in_class and brace_count == 0:
                end_index = i
                break
                # if few_shot and num_complete < 1:
                #     num_complete += 1
                # else: 
                #     end_index = i
                #     break

    if end_index is not None:
        return code[:end_index + 1]
    else:
        return code[:]  # fallback if not closed properly

code = r"""//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public double triangleArea(double a, double h) {
        return a * h / 2;
    }
}

//Unit Test Cases
public class Main {
    public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.triangleArea(5, 3) == 7.5,
                s.triangleArea(2, 2) == 2.0,
                s.triangleArea(10, 8) == 40.0
        );
        if (correct.contains(false)) {
            throw new AssertionError();
        }
    }
}


//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public int strlen(String string) {
        return string.length();
    }
}

//Unit Test Cases
public class Main {
        public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.strlen(""Hello"") == 5,
                s.strlen(""World"") == 5,
                s.strlen(""Java"") == 4
        );
        if (correct.contains(false)) {
            throw new AssertionError();
        }
    }
}


//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public int strlen(String string) {
        return string.length();
    }
}

//Unit Test Cases
public class Main {
        public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.strlen(""Hello"") == 5,
                s.strlen(""World"") == 5,
                s.strlen(""Java"") == 4
        );
        if (correct.contains(false)) {
            throw new AssertionError();
        }
    }
}


//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public int strlen(String string) {
        return string.length();
    }
}

//Unit Test Cases
public class Main {
        public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.strlen(""Hello"") ==" """

print(extract_main_class(code, few_shot=False))
# import pandas as pd
# df = pd.read_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__nick/responses.csv")

# # extract_main_class(df.iloc[0].iloc[0], few_shot=True)

# trimmed = []
# for row in df:
#     untrimmed_response = row
#     trimmed_response = extract_main_class(untrimmed_response, few_shot=True)
#     trimmed.append(trimmed_response)

# import os
# os.mkdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses")
# df.to_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses/few_shot_java.csv")



//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public double triangleArea(double a, double h) {
        return a * h / 2;
    }
}

//Unit Test Cases
public class Main {
    public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.triangleArea(5, 3) == 7.5,
                s.triangleArea(2, 2) == 2.0,
                s.triangleArea(10, 8) == 40.0
        );
        if (correct.contains(false)) {
            throw new AssertionError();
        }
    }
}


In [25]:
import re

def extract_main_class(code: str, few_shot: bool) -> str:
    # Determine which instance to extract (2nd if few-shot, 1st otherwise)
    n = 2 if few_shot else 1

    # Find all indices where "public class Main" appears
    starts = [m.start() for m in re.finditer(r'public class Main', code)]
    
    if len(starts) < n:
        return ""  # Not enough occurrences

    start_index = starts[n - 1]  # Get the nth occurrence (1-based index)

    # Brace counting to find the matching closing }
    brace_count = 0
    in_class = False
    end_index = None

    for i in range(start_index, len(code)):
        char = code[i]
        if char == '{':
            brace_count += 1
            if not in_class:
                in_class = True
        elif char == '}':
            brace_count -= 1
            if in_class and brace_count == 0:
                end_index = i
                break

    # Return only the full class block, starting from the nth "public class Main"
    if end_index is not None:
        return code[start_index:end_index + 1].strip()
    else:
        return code[start_index:].strip()  # Fallback if closing brace is missing

In [ ]:
import pandas as pd
df = pd.read_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__nick/responses.csv")

# extract_main_class(df.iloc[0].iloc[0], few_shot=True)

trimmed = []
for row in df:
    untrimmed_response = row.iloc[0]
    trimmed_response = extract_main_class(untrimmed_response, few_shot=False)
    trimmed.append(trimmed_response)

import os
os.mkdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses")
df.to_csv("/media/mujtaba/DATA/nick/UnitTestExamples/data/clean_responses/no_shot_java.csv")

In [ ]:
df = pd.read_csv("")

In [37]:
df.iloc[3]['code']

'//Source method\nimport java.util.*;\nimport java.lang.*;\n\nclass Solution {\n    public double truncateNumber(double number) {\n        return number % 1.0;\n    }\n}\n\n//Unit Test Cases\npublic class Main {\n        public static void main(String[] args) {\n            Solution solution = new Solution();\n            double number = 123.456;\n            double result = solution.truncateNumber(number);\n            System.out.println(result);\n        }\n}\n/src/main/java/com/leetcode/easy/easy_1001_1100/easy_1001_1010/easy_1001_1005/Solution1005.java\n//Source method\nclass Solution {\n    public int largestSumAfterKNegations(int[] A, int K) {\n        Arrays.sort(A);\n        int sum = 0;\n        for (int i = 0; i < A.length; i++) {\n            if (A[i] < 0) {\n                if (K % 2 == 0) {\n                    A[i] = -A[i];\n                } else {\n                    if (i == 0 || A[i - 1] > 0) {\n                        A[i] = -A[i];\n                    }\n          

In [45]:
files = []

import os

for file in os.listdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/results"):
    if "CodeLlama" in file or "starcoder" in file:
        files.append('/' + file)

files = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results" + file + '/responses_reformatted.csv' for file in files]
print(files)

import pandas as pd



for file in files:
    df = pd.read_csv(file, quoting=1)
    if "few_shot" in file:
        few_shot = True
    else:
        few_shot = False

    for i in range(len(df)):
        if i==0:
            continue
        print(df.iloc[i]['code'])
        untrimmed_response = df.iloc[i]['code']
        trimmed_response = extract_main_class(untrimmed_response, few_shot=few_shot)
        # trimmed.append(trimmed_response)   
        os.makedirs(f"/media/mujtaba/DATA/nick/UnitTestExamples/data/results/{file.split('/')[-2]}/java_files", exist_ok = True)
        output_path = os.path.join(f"/media/mujtaba/DATA/nick/UnitTestExamples/data/results/{file.split('/')[-2]}/java_files/", f"{i-1}.java")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(trimmed_response)

['/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/responses_reformatted.csv', '/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/responses_reformatted.csv', '/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__zero_shot_first_prompts/responses_reformatted.csv', '/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__zero_shot_first_prompts/responses_reformatted.csv']
//Source method
import java.util.*;
import java.lang.*;

class Solution {
    public double triangleArea(double a, double h) {
        return a * h / 2;
    }
}

//Unit Test Cases
public class Main {
    public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.triangleArea(5, 3) == 7.5,
                s.triangleArea(2, 2) == 2.0,
                s.triangleArea(10, 8) == 40.0
        );
        if 

## Removing Few Shot Prompts

In [64]:
import re

def remove_first_solution_and_test(java_file_path, output_path):
    with open(java_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Use regex to split on Source method and keep separators
    chunks = re.split(r'(//Source method)', content)
    # for chunk in chunks:
    #     # print(chunk + '\nBREAK\n')
    print(chunks[4] + "\n\nDONE\n\n")

    # # Reconstruct full chunks with their headers (//Source method)
    # combined = []
    # for i in range(1, len(chunks), 2):
    #     combined.append(chunks[i] + chunks[i+1])

    # Each example is a pair of Source + Unit Test
    # # So skip the first two blocks (index 0 and 1)
    # # and keep everything else
    # trimmed = ''.join(combined[2:])  # Skip first Source and Unit Test block'AST

    trimmed = chunks[4]

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(trimmed)

    # print(f"Trimmed Java code saved to: {output_path}")


In [66]:
import os

directories = []

for directory in os.listdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/results"):
    if "CodeLlama" in directory or "starcoder" in directory:
        if "few_shot" in directory:
            directories.append('/' + directory)

directories = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results" + directory + '/java_files' for directory in directories]
print(directories)
for directory in directories:
    os.makedirs(directory + '/trimmed', exist_ok=True)
    for i in range(164):
        remove_first_solution_and_test(directory + f'/{i}.java', directory + '/trimmed' + f'/{i}.java')



['/media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files', '/media/mujtaba/DATA/nick/UnitTestExamples/data/results/starcoder2-3b__few_shot_first_prompts/java_files']

import java.util.*;
import java.lang.*;

class Solution {
    public boolean hasCloseElements(List<Double> numbers, double threshold) {
        for (int i = 0; i < numbers.size(); i++) {
            for (int j = i + 1; j < numbers.size(); j++) {
                double distance = Math.abs(numbers.get(i) - numbers.get(j));
                if (distance < threshold) return true;
            }
        }
        return false;
    }
}

//Unit Test Cases
public class Main {
        public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.hasCloseElements(Arrays.asList(1.0, 2.0, 3.0, 4.0, 5.0), 1.5) == true,
                s.hasCloseElements(Arrays.asList(1.0, 2.0, 3.0, 4.0, 5.0), 1.0) 

## Separating Generated Tests From Source Method

In [71]:
import os
import re

def extract_main_class_only(code):
    """
    Extracts everything from '//Unit Test Cases' or 'public class Main' onward.
    Does not require balanced braces, to preserve incomplete generations.
    """
    # Try to find from the //Unit Test Cases comment
    match = re.search(r'//Unit Test Cases\s*(public class Main\s*{.*)', code, re.DOTALL)
    if match:
        return match.group(1)
    
    # If no comment marker, fall back to just 'public class Main'
    match = re.search(r'(public class Main\s*{.*)', code, re.DOTALL)
    if match:
        return match.group(1)
    
    print("Warning: Could not find 'public class Main' in input.")
    return ""

def extract_all_mains(source_dir):
    output_dir = os.path.join(source_dir, "generations")
    os.makedirs(output_dir, exist_ok=True)

    for i in range(0, 164):  # 0.java through 163.java
        file_path = os.path.join(source_dir, f"{i}.java")
        output_path = os.path.join(output_dir, f"{i}.java")

        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue

        with open(file_path, "r", encoding="utf-8") as f:
            code = f.read()

        main_code = extract_main_class_only(code)

        if main_code.strip():
            with open(output_path, "w", encoding="utf-8") as out:
                out.write(main_code)
            print(f"Saved: {output_path}")
        else:
            print(f"Skipping (empty or missing Main): {file_path}")


import os
directories = []

for directory in os.listdir("/media/mujtaba/DATA/nick/UnitTestExamples/data/results/"):
    if "CodeLlama" in directory or "starcoder" in directory:
        if "few_shot" in directory:
            print(directory)
            directories.append("/" + directory + "/java_files/trimmed/")
        else:
            directories.append("/" + directory + "/java_files/")

directories = ["/media/mujtaba/DATA/nick/UnitTestExamples/data/results" + directory for directory in directories]

for directory in directories:
    extract_all_mains(directory)

CodeLlama-7b-Instruct-hf__few_shot_first_prompts
starcoder2-3b__few_shot_first_prompts
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/0.java
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/1.java
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/2.java
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/3.java
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/4.java
Saved: /media/mujtaba/DATA/nick/UnitTestExamples/data/results/CodeLlama-7b-Instruct-hf__few_shot_first_prompts/java_files/trimmed/generations/5.java
Saved: /media/mujta